## 0. Package Loading

In [1]:
import pandas as pd
pd.options.display.max_columns = 500

import pickle

import numpy as np


# import scipy.sparse as sp
# from scipy.sparse.linalg import svds
# from sklearn.decomposition import NMF
# from sklearn.datasets import dump_svmlight_file
# from scipy.sparse import csr_matrix, find


## I. Importing the data

In [11]:
hetrec2011_dat_files = !ls ../Resources/Data/Raw/hetrec2011-movielens-2k-v2/
hetrec2011_dat_files.remove("readme.txt")
hetrec2011_dat_files_pwd = []
hetrec2011_table_names = []
for i in range(len(hetrec2011_dat_files)):
    # This creates a new item in the hetrec2011_dat_files_pwd list which has the full directory path
    hetrec2011_dat_files_pwd.append("../Resources/Data/Raw/hetrec2011-movielens-2k-v2/" + hetrec2011_dat_files[i])
    # This distills out just the names of the taples (without "dat" at the end)
    hetrec2011_table_names.append(hetrec2011_dat_files[i][:-4])

# hetrec2011_dat_files_pwd
# hetrec2011_table_names
# hetrec2011_table_names    

In [12]:
dictionary_of_hetrec_dataframes = {}
n = len(hetrec2011_dat_files_pwd)

for i in range(n):
    table_name = hetrec2011_table_names[i]
    table_path = hetrec2011_dat_files_pwd[i]
    dictionary_of_hetrec_dataframes[table_name] = pd.read_table(table_path)

dictionary_of_hetrec_dataframes["movies"].rename(columns={'id':'movieID'}, inplace=True)
dictionary_of_hetrec_dataframes["tags"].rename(columns={'id':'tagID'}, inplace=True)
dictionary_of_hetrec_dataframes["tags"].rename(columns={'value':'tag_value'}, inplace=True)

In [13]:
dictionary_of_hetrec_dataframes.keys()

['movie_actors',
 'user_ratedmovies-timestamps',
 'movie_locations',
 'movie_directors',
 'tags',
 'user_taggedmovies',
 'user_taggedmovies-timestamps',
 'movies',
 'movie_genres',
 'movie_countries',
 'movie_tags',
 'user_ratedmovies']

In [14]:
from PIL import Image
import requests
import numpy as np
from StringIO import StringIO

import PIL.Image
from cStringIO import StringIO
import IPython.display

In [15]:
def showarray(a, fmt='png'):
    """This Function Takes a NP array image and displays it."""
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

#### Getting a dictionary of movie poster information: # Key: ID Value:( title, img url, shape tuple, image )

In [16]:
path = '../Resources/Data/Raw/hetrec2011-movielens-2k-v2/movies.dat'
movies_df = pd.read_table(path)

list_of_movieIDs = movies_df["id"]
list_of_movieTitles = movies_df["title"]
list_of_urls = movies_df["imdbPictureURL"]

In [19]:
print len(list_of_movieIDs)

10197


In [48]:
# Commenting this out since it's run and and then has been pickled.

poster_dictionary = {}
# Key: ID Value:( title, img url, local image path)

for i in range(len(list_of_movieIDs)):
#for i in range(24):
    if i in poster_dictionary:  #Basically, if i've already added that movie into the dictionary, move onto the next movie
        pass
    if i % 500 == 0: 
        print i,
    movieID = list_of_movieIDs[i]
    title = list_of_movieTitles[i]
    #print title
    url = list_of_urls[i]
    #print url
    if type(url) == float:   #This occurs if the cell has "nan" inside of it. E.g. nan_error = list_of_urls[134]
        #print "nan value", i,
        #img_as_array = float("NaN")
        image_path = "NaN"
    else:
        response = requests.get(url)
        #print response
        #print type(response)
        if response.content == "":   #This is the "content" feature of an instance of a bad_404_response = requests.get(list_of_urls[i])
            #print "bad response", i, 
            #img_as_array = float("NaN")
            image_path = "NaN"
        else:            
            image = Image.open(StringIO(response.content))
            #img_as_array = np.array(Image.open(StringIO(response.content)))
            image_path = "../Resources/Data/Raw/MoviePosters/" + str(movieID) + ".jpg"
            
            image.save(image_path)
                               
#######################

    #showarray(img)
    poster_dictionary[movieID] = (title,url,image_path)

0 500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000


In [55]:
# with open('../Resources/Data/Pickles/poster_dictionary.pickle', 'wb') as handle:
#   pickle.dump(poster_dictionary, handle)

with open('poster_dictionary.pickle', 'rb') as handle:
  poster_dictionary = pickle.load(handle)

In [57]:
print poster_dictionary[25][0]


Leaving Las Vegas


In [63]:
MoviePosters_df = pd.DataFrame(poster_dictionary).T

In [64]:
MoviePosters_df.columns = ["MovieTitle","OnlinePosterLink","LocalPosterPath"]
MoviePosters_df.head()

,MovieTitle,OnlinePosterLink,LocalPosterPath
1,Toy story,http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,../Resources/Data/Raw/MoviePosters/1.jpg
2,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,../Resources/Data/Raw/MoviePosters/2.jpg
3,Grumpy Old Men,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,../Resources/Data/Raw/MoviePosters/3.jpg
4,Waiting to Exhale,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,../Resources/Data/Raw/MoviePosters/4.jpg
5,Father of the Bride Part II,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,../Resources/Data/Raw/MoviePosters/5.jpg


In [65]:
MoviePosters_df.to_csv("../Resources/Data/Derived/MoviePosters_df.csv")

#### Using the dictionary to create image features --> img to serieal numbers / array

Convert multidimensional array to single dimension array / image vector